In [5]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from flask import Flask, request, render_template_string
import io, base64


def get_returns(tickers): # μ
    expected_returns = []

    for t in tickers:
        data = yf.download(t, start="2009-12-01", end="2025-01-04", interval="1mo", auto_adjust=True, progress=False)["Close"]
        #very important- the prices that we get are the closing prices for each month (even if in the dataframe it says day 1 
        #of the month)
        #And the end of the data frame is the close of the month before the month of our end date
        years = int((len(data)-1)/12)

        data.index = pd.to_datetime(data.index)
        mask = (data.index.month == 12)& (data.index.day == 1)
        first_idx = data.index[mask][0]
        data = data.loc[first_idx:].reset_index(drop=True)
        #we need to make sure we are staring at the close of a december (some companies didnt have their IPO by our start date)

        anual_returns = []
        for i in range(years):
            anual_return = round(((data.iloc[(i+1)*12].item()- data.iloc[i*12].item())/data.iloc[i*12].item()), 4)
            anual_returns.append(anual_return)
        mean_return = round(np.mean(anual_returns).item(),2)
        expected_returns.append(mean_return)
        #IMPORTANT - we consider a anual return the change between the close price of the certain year and the close 
        # price of the year before --- other sources might do the open of a certain year and the close of that year.

    return expected_returns

def get_volatilities(tickers): #σ  
  volatilities = []
  for t in tickers:
      data = yf.download(t, start="2009-12-31", end="2025-01-01", interval="1d", auto_adjust=True, progress=False)["Close"]
      #as we get the closing prices, the starting point need to be the close before the 1st day we want to get the return to
      #and the last close price we get is the close before our end date
      trading_days = 252 #small rounding...
      data.index = pd.to_datetime(data.index)
      mask = (data.index.month == 12)& (data.index.day == 31)
      first_idx = data.index[mask][0]
      data = data.loc[first_idx:].reset_index(drop=True)
      #we need to make sure we are staring at the close of a december (some companies didnt have their IPO by our start date)
      daily_returns=[]
      for i in range(1,len(data)):
          d_return=round(((data.iloc[i].item()-data.iloc[i-1].item())/data.iloc[i-1].item()),4)
          daily_returns.append(d_return)
      sigma = round((np.std(daily_returns, ddof=1))*np.sqrt(trading_days),2).item() #annualized standard deviation (of the daily returns)
      volatilities.append(sigma)

  return volatilities

def covariance_matrix(tickers):
    all_returns = []
    for t in tickers:
        data = yf.download(t, start="2009-12-31", end="2025-01-01", interval="1d", auto_adjust=True, progress=False)["Close"]
        data.index = pd.to_datetime(data.index)
        mask = (data.index.month == 12)& (data.index.day == 31)
        first_idx = data.index[mask][0]
        data = data.loc[first_idx:].reset_index(drop=True)
        #we need to make sure we are staring at the close of a december (some companies didnt have their IPO by our start date)
        daily_returns = []
        for i in range(1,len(data)):
            d_return=round(((data.iloc[i].item()-data.iloc[i-1].item())/data.iloc[i-1].item()),4)
            daily_returns.append(d_return)
        all_returns.append(daily_returns)

    #now we make sure that all the lists of daily returns in the list of lists have the same size (but cutting the exess in the bigger ones)
    min_len = min(len(i) for i in all_returns)
    for i in all_returns:
        all_returns[all_returns.index(i)] = i[-min_len:]

    matrix_of_returns = np.column_stack([np.asarray(x) for x in all_returns]) #each collumn is a stock
    covariance_matrix = np.cov(matrix_of_returns, rowvar=False, ddof=1)*252 #we multiply by 252 to anualize

    return covariance_matrix

def compute_benchmark(symbol,rf):
        data = yf.download(symbol, start="2009-12-01", end="2025-01-04", interval="1mo", auto_adjust=True, progress=False)["Close"]
        #very important- the prices that we get are the closing prices for each month (even if in the dataframe it says day 1 
        #of the month)
        #And the end of the data frame is the close of the month before the month of our end date
        years = int((len(data)-1)/12)

        data.index = pd.to_datetime(data.index)
        mask = (data.index.month == 12)& (data.index.day == 1)
        first_idx = data.index[mask][0]
        data = data.loc[first_idx:].reset_index(drop=True)
        #we need to make sure we are staring at the close of a december (some companies didnt have their IPO by our start date)
        anual_returns = []
        for i in range(years):
            anual_return = round(((data.iloc[(i+1)*12].item()- data.iloc[i*12].item())/data.iloc[i*12].item()), 4)
            anual_returns.append(anual_return)
        mean_return = round(np.mean(anual_returns).item(),2)

        data = yf.download(symbol, start="2009-12-31", end="2025-01-01", interval="1d", auto_adjust=True, progress=False)["Close"]
        #as we get the closing prices, the starting point need to be the close before the 1st day we want to get the return to
        #and the last close price we get is the close before our end date
        trading_days = 252 #small rounding...
        data.index = pd.to_datetime(data.index)
        mask = (data.index.month == 12)& (data.index.day == 31)
        first_idx = data.index[mask][0]
        data = data.loc[first_idx:].reset_index(drop=True)
        #we need to make sure we are staring at the close of a december (some companies didnt have their IPO by our start date)
        daily_returns=[]
        for i in range(1,len(data)):
            d_return=round(((data.iloc[i].item()-data.iloc[i-1].item())/data.iloc[i-1].item()),4)
            daily_returns.append(d_return)
        sigma = round((np.std(daily_returns, ddof=1))*np.sqrt(trading_days),2).item() #annualized standard deviation (of the daily returns)
        shapre = round((mean_return - rf)/sigma,2)

        return(mean_return, sigma, shapre)


   


def portfolios (tickers, wanted_return, wanted_volatility, rf):
  return_vector =  np.array(get_returns(tickers)).reshape(-1, 1)
  sigma_matrix = covariance_matrix(tickers)

  #Now we will do the lagrangian to find the vector of weights of each stock in the portfolio w = (w1, w2, w3, ... , wn) that minimize
  #the variance of the porfolio for a given wanted return

  #We know, from the Lagrangian, that the optimal portfolio is a combination of 2 portfolions
  #The portfolio of minimum variance, and a second portfolio
  ones = np.ones((len(return_vector), 1))

  A = (ones.T @ np.linalg.inv(sigma_matrix) @ ones).item()
  B = (ones.T @ np.linalg.inv(sigma_matrix) @ return_vector).item()
  C = (return_vector.T @ np.linalg.inv(sigma_matrix) @ return_vector).item()
  DELTA =(A*C - B**2)



  minimum_variance_portfolio = (np.linalg.inv(sigma_matrix) @ ones)/A
  second_portfolio = (np.linalg.inv(sigma_matrix) @ return_vector)/B

  return_minumim_variance_portfolio = (minimum_variance_portfolio.T @ return_vector).item()
  sd_minimum_variance_portfolio = (np.sqrt((A*(return_minumim_variance_portfolio**2)-2*B*return_minumim_variance_portfolio + C)/DELTA)).item()
  sharpe_minimum_variance_portfolio = (return_minumim_variance_portfolio - rf)/sd_minimum_variance_portfolio
  #this is where we condition the code on weather we have a target return or a target volatility
  if wanted_return == "blank": #only one can be blank
      if wanted_volatility<sd_minimum_variance_portfolio:
          r_for_no_rf = return_minumim_variance_portfolio
      else:
          r_for_no_rf = max((2*B+np.sqrt(4*B**2-4*A*(C-DELTA*wanted_volatility**2)))/(2*A), (2*B-np.sqrt(4*B**2-4*A*(C-DELTA*wanted_volatility**2)))/(2*A))
      
      r_with_rf = max(wanted_volatility*np.sqrt(C-2*rf*B+(rf**2)*A)+rf , -(wanted_volatility*np.sqrt(C-2*rf*B+(rf**2)*A))+rf)

  else:
      if wanted_return < return_minumim_variance_portfolio:
        r_for_no_rf = return_minumim_variance_portfolio
      else:
         r_for_no_rf = wanted_return
      r_with_rf = wanted_return


  #No Risk-free Asset

  optimal_portfolio =(((C-r_for_no_rf*B)*A)/DELTA)*minimum_variance_portfolio + (((r_for_no_rf*A-B)*B)/DELTA)*second_portfolio
  sd_optimal_portfolio =np.sqrt((A*(r_for_no_rf**2)-2*B*r_for_no_rf + C)/DELTA).item()
  sharpe_optimal = (r_for_no_rf-rf)/sd_optimal_portfolio

  r = np.linspace(0 , max(get_returns(tickers))+0.1, 1000)
  sd_general = np.sqrt((A*(r**2)-2*B*r + C)/DELTA)

  #RISK FREE ASSET

  #tagent portfolio - weights of the risky assets

  tangent_portfolio = (np.linalg.inv(sigma_matrix)@(return_vector-rf*ones))/(B-rf*A)
  return_tangent_portfolio = (tangent_portfolio.T @ return_vector).item()
  sd_tangent_portfolio = (np.sqrt((A*(return_tangent_portfolio**2)-2*B*return_tangent_portfolio + C)/DELTA)).item()
  #sd_tagent_portfolio2 = (abs(return_tagent_portfolio-rf))/np.sqrt(C-2*rf*B+(rf**2)*A)
  # - both ways of calculating sd work because this portfolio is in both lines
  sharpe_tangent = (return_tangent_portfolio-rf)/sd_tangent_portfolio

  #The optimal portfolio with a riskless asset is a combination of the tagent portfolio and a riskless asset
  lagrangian_multiplier_riskless = (r_with_rf - rf)/(C-2*rf*B+(rf**2)*A)

  optimal_portfolio_w_rf_riskyweights = lagrangian_multiplier_riskless*(B-rf*A)*tangent_portfolio
  optimal_portfolio_w_rf_leverage = 1 - optimal_portfolio_w_rf_riskyweights.T@ones

  sd_optimal_portfolio_w_riskless = (abs(r_with_rf-rf))/np.sqrt(C-2*rf*B+(rf**2)*A).item()

  #graph for the lines that are the combinations of tagent portfolio with the risk free asset
  sd_general_for_w_riskfree = (abs(r-rf))/np.sqrt(C-2*rf*B+(rf**2)*A)
  everything = [minimum_variance_portfolio, return_minumim_variance_portfolio, sd_minimum_variance_portfolio,
                optimal_portfolio, sd_optimal_portfolio,
                r, sd_general,
                tangent_portfolio, return_tangent_portfolio, sd_tangent_portfolio,
                optimal_portfolio_w_rf_riskyweights, optimal_portfolio_w_rf_leverage, sd_optimal_portfolio_w_riskless,
                sd_general_for_w_riskfree,
                r_for_no_rf, r_with_rf,
                sharpe_minimum_variance_portfolio, sharpe_optimal, sharpe_tangent]
  return(everything)


def graph(tickers, wanted_return, wanted_volatility, rf, view="mvp", benchmark=None, show_tickers = True, benchmark_label=None):

    everything = portfolios(tickers, wanted_return, wanted_volatility, rf)

    fig = plt.figure(figsize=(9, 5.4), dpi=120)
    plt.gcf().patch.set_alpha(0)           # figure background
    plt.gca().set_facecolor('none')        # axes background

    plt.rcParams['text.color'] = '#e5e7eb'
    plt.rcParams['axes.labelcolor'] = '#e5e7eb'
    plt.rcParams['xtick.color'] = '#e5e7eb'
    plt.rcParams['ytick.color'] = '#e5e7eb'

    for side in ['top', 'right', 'bottom', 'left']:
        plt.gca().spines[side].set_color('#cbd5e1')

    plt.tick_params(colors='#e5e7eb', which='both')
    plt.grid(True, color='#94a3b8', alpha=0.35, linewidth=0.8)
    plt.gca().set_axisbelow(True)

    v = (view or "mvp").lower()
    if isinstance(benchmark, (tuple, list)) and len(benchmark) == 2:
        x, y = benchmark[1], benchmark[0]
        label = "Benchmark"
        if benchmark_label:
            label = f"{label} ({benchmark_label})"
        plt.scatter([x], [y], alpha=0.9)
        plt.annotate(label, (x, y), (4, 0), textcoords='offset points')
    if v == "mvp":
      plt.plot(everything[6], everything[5])
      plt.scatter([everything[2]],[everything[1]])
      plt.annotate(f"Minimum Variance Portfolio",
                  (everything[2], everything[1]),
                  (4, 0), textcoords='offset points')
    elif v == "tangent":
      plt.plot(everything[6], everything[5])
      plt.scatter([everything[9]], [everything[8]])
      plt.annotate(f"Tangent Portfolio",
                  (everything[9], everything[8]),
                  (4, 0), textcoords='offset points')
    elif v == "opt_norf":
      plt.plot(everything[6], everything[5])
      plt.scatter([everything[9]], [everything[8]])
      plt.annotate(f"Tangent Portfolio",
                  (everything[9], everything[8]),
                  (4, 0), textcoords='offset points')
      plt.scatter([everything[4]], [everything[14]])
      plt.annotate(f"Optimal Portfolio",
                    (everything[4], everything[14]),
                    (-90, 0), textcoords='offset points')
    elif v == "opt_rf":
      plt.plot(everything[6], everything[5], linestyle= "--")
      plt.plot(everything[13], everything[5])
      plt.scatter([everything[9]], [everything[8]])
      plt.annotate(f"Tangent Portfolio",
                  (everything[9], everything[8]),
                  (4, 0), textcoords='offset points')
      plt.scatter([everything[12]],[everything[15]])
      plt.annotate(f"Optimal Portfolio + Riskless Asset",
                    (everything[12], everything[15]),
                    (-170, 0), textcoords='offset points')      
    if show_tickers:
      for t in range(len(tickers)):
          plt.scatter([get_volatilities(tickers)[t]],[get_returns(tickers)[t]], alpha= 0.6)
          plt.annotate(tickers[t],
                      (get_volatilities(tickers)[t],get_returns(tickers)[t]),
                      (4, 0), textcoords='offset points', alpha = 0.6)
    plt.xlabel('Volatility σ', color='#e5e7eb')
    plt.ylabel('Return μ', color='#e5e7eb')

    plt.gca().relim()
    plt.gca().autoscale_view()
    ymin, ymax = plt.ylim()
    plt.ylim(0, ymax * 1.05)   # 5% de folga acima do maior retorno

    plt.tight_layout(pad=1.1)
    #plt.close(fig)

    return(fig)

app = Flask(__name__)


PAGE = """
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <title>Portfolio Optimization</title>
  <style>
    :root{
      --bg:#0f172a;
      --text:#e5e7eb;
      --muted:#a3a8b8;
      --accent:#60a5fa;
      --accent-2:#22d3ee;
      --border:rgba(255,255,255,.10);
    }

    *{ box-sizing:border-box }
    html,body{ height:100% }
    body{
      margin:24px;
      font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
      color:var(--text);
      background:
        radial-gradient(1100px 680px at 10% -10%, #1e293b 0%, rgba(30,41,59,0) 60%),
        radial-gradient(900px 600px at 100% 0%, #112040 0%, rgba(17,32,64,0) 60%),
        var(--bg);
    }

    /* Page title + signature */
    h1{ margin:0 0 6px 0; text-align:center; font-weight:800; letter-spacing:.4px; }
    .subtitle{ text-align:center; margin:0 0 22px 0; color:var(--muted); font-size:14px; }

    .card{
      background: transparent;
      border: none;
      box-shadow: none;
      padding: 0 0 16px 0; /* only spacing */
      margin: 14px 0;
    }

    label{ font-weight:600; display:block; margin: 10px 0 6px; color:var(--muted); }

    /* Inputs — dark style */
    input[type=text], input[type=number]{
      width:100%; padding:12px;
      border:1px solid var(--border); border-radius:10px;
      background:#0b1221; color:var(--text);
      outline:none; transition:border-color .2s, box-shadow .2s;
    }
    input::placeholder{ color:#808aa0 }
    input:focus{ border-color:rgba(96,165,250,.6); box-shadow:0 0 0 3px rgba(96,165,250,.15); }

    /* Layout grids */
    .row{ display:grid; grid-template-columns: 1fr 1fr; gap:16px; }
    .row-views{
      display:grid; grid-template-columns: repeat(4, auto);
      gap:26px; margin: 18px 6px 8px; justify-content:center;
      border-bottom:1px solid var(--border); padding-bottom:6px;
    }
    .two-col{ display:grid; grid-template-columns: minmax(320px, 1fr) 2fr; gap:22px; align-items:start; }
    @media (max-width: 980px){
      .two-col{ grid-template-columns:1fr; }
      .row{ grid-template-columns:1fr; }
      .row-views{ grid-template-columns: repeat(2, auto); }
    }

    /* “Buttons” styled as top nav links (no boxes) */
    .btn{
      appearance:none; background:transparent; border:none; cursor:pointer;
      color:var(--muted); padding:10px 2px; font-weight:700; letter-spacing:.2px;
      position:relative;
    }
    .btn:hover{ color:#fff; }
    .btn.active{ color:#fff; }
    .btn.active::after, .btn:hover::after{
      content:""; position:absolute; left:0; right:0; bottom:-8px; height:2px;
      background: linear-gradient(90deg, var(--accent), var(--accent-2)); border-radius:2px;
    }

    /* Table — minimal separators, no box */
    table{ border-collapse:collapse; width:100%; }
    th, td{ padding:8px 10px; text-align:right; border-bottom:1px dashed var(--border); }
    th:first-child, td:first-child{ text-align:left; }
    thead th{ color:#c7d2fe; font-weight:700; border-bottom:1px solid var(--border); }
    h3{ margin:8px 0 10px }

    small{ color:var(--muted); }
    .err{ color:#ef4444; font-weight:700; }

    /* Chart header + toggle + benchmark chip */
    .card-head{ display:flex; align-items:center; justify-content:space-between; gap:12px; margin:12px 0 10px; }
    .bm-sr{
      font-size:13px; color:#dbeafe; background:rgba(96,165,250,.12);
      border:1px solid rgba(96,165,250,.35); padding:6px 10px; border-radius:999px; white-space:nowrap;
    }

    /* Toggle */
    .toggle{ display:inline-flex; align-items:center; gap:8px; font-size:14px; color:var(--muted); }
    .switch{ position:relative; display:inline-block; width:40px; height:22px; }
    .switch input{ opacity:0; width:0; height:0; }
    .slider{ position:absolute; inset:0; cursor:pointer; background:#202a44; transition:.2s; border-radius:999px; border:1px solid var(--border); }
    .slider:before{ content:""; position:absolute; height:16px; width:16px; left:3px; top:3px; background:#fff; border-radius:50%; transition:.2s; }
    .switch input:checked + .slider{ background: linear-gradient(90deg, var(--accent), var(--accent-2)); border-color:transparent; }
    .switch input:checked + .slider:before{ transform:translateX(18px); }

    /* Chart image — no border/box */
    .plot{ width:100%; height:auto; border:none; border-radius:12px; background:transparent; }

    .plot-wrap { position: relative; }
    .coord-badge{
      position: absolute; top: 8px; left: 8px;
      font-size: 12px; color: #dbeafe; background: rgba(96,165,250,.14);
      border: 1px solid rgba(96,165,250,.35); padding: 6px 8px; border-radius: 8px;
      pointer-events: none; opacity: 0; transition: opacity .15s;
    }
    .plot-wrap:hover .coord-badge{ opacity: 1; }
  </style>
</head>
<body>

  <h1>Portfolio Optimization</h1>
  <div class="subtitle">by Tomás Gil Mata</div>

  <div class="card">
    <form method="post">
      <label>Tickers (yfiance format - comma separated)</label>
      <input type="text" name="tickers" placeholder="AAPL, MSFT, KO" value="{{ request.form.get('tickers','') }}">

      <div class="row">
        <div>
          <label>Desired return μ(%) (leave blank if you set σ)</label>
          <input type="text" name="wanted_return" value="{{ request.form.get('wanted_return','') }}">
        </div>
        <div>
          <label>Desired volatility σ(%) (leave blank if you set μ)</label>
          <input type="text" name="wanted_volatility" value="{{ request.form.get('wanted_volatility','') }}">
        </div>
      </div>

      <div class="row">
        <div>
          <label>Risk-free rate (%)</label>
          <input type="text" name="rf" value="{{ request.form.get('rf','') }}">
        </div>
        <div>
          <label>Benchmark Ticker (yfinance format - optional)</label>
          <input type="text" name="benchmark" placeholder="^GSPC" value="{{ request.form.get('benchmark','') }}">
        </div>
      </div>

      <p><small>Provide <b>either</b> desired μ or desired σ (not both).</small></p>

      {% set v = (results.view if results and ('view' in results) else 'mvp') %}
      <div class="row-views">
        <button class="btn {% if results and v=='mvp' %}active{% endif %}" type="submit" name="view" value="mvp" aria-pressed="{{ 'true' if v=='mvp' else 'false' }}">Minimum Variance</button>
        <button class="btn {% if results and v=='tangent' %}active{% endif %}" type="submit" name="view" value="tangent" aria-pressed="{{ 'true' if v=='tangent' else 'false' }}">Tangent</button>
        <button class="btn {% if results and v=='opt_norf' %}active{% endif %}" type="submit" name="view" value="opt_norf" aria-pressed="{{ 'true' if v=='opt_norf' else 'false' }}">Optimal (no riskless asset)</button>
        <button class="btn {% if results and v=='opt_rf' %}active{% endif %}" type="submit" name="view" value="opt_rf" aria-pressed="{{ 'true' if v=='opt_rf' else 'false' }}">Optimal (w/riskless asset)</button>
      </div>

      {% if error %}
        <div class="card" style="padding:0;"><span class="err">Error:</span> {{ error }}</div>
      {% endif %}

      {% if results %}
        {% macro weights_table(title, rows, r, s, sp) -%}
          <div class="card" style="padding:0;">
            <h3>{{ title }}</h3>
            <table>
              <thead><tr><th>Ticker</th><th>Weight</th></tr></thead>
              <tbody>
                {% for t, w in rows %}
                  <tr><td>{{ t }}</td><td>{{ '%.2f%%' % (100*w) }}</td></tr>
                {% endfor %}
              </tbody>
            </table>
            <p><b>Return μ:</b> {{ '%.2f%%' % (100*r) }} &nbsp;&nbsp; <b>Vol σ:</b> {{ '%.2f%%' % (100*s) }}</p>
            <p><b>Sharpe Ratio:</b> {{ '%.3f' % sp }}</p>
          </div>
        {%- endmacro %}

        <div class="two-col" style="margin-top:10px;">
          <div>
            {% if v == "mvp" %}
              {{ weights_table("Minimum Variance Portfolio", results.mvp_weights, results.mvp_r, results.mvp_s, results.mvp_sp) }}
            {% elif v == "tangent" %}
              {{ weights_table("Tangent Portfolio", results.tangent_weights, results.tangent_r, results.tangent_s, results.tp_sp) }}
            {% elif v == "opt_norf" %}
              {{ weights_table("Optimal (no risk-free)", results.opt_norf_weights, results.opt_norf_r, results.opt_norf_s, results.op_sp) }}
            {% elif v == "opt_rf" %}
              {{ weights_table("Optimal (+ risk-free)", results.opt_rf_weights, results.opt_rf_r, results.opt_rf_s, results.tp_sp) }}
            {% endif %}
          </div>

          <div>
            <div class="card" style="padding:0;">
              <div class="card-head">
                <h3 style="margin:0;">Portfolio Frontier</h3>

                {% if results and (results.bm_sp is not none) %}
                  <span class="bm-sr">Benchmark Sharpe Ratio: {{ '%.3f' % results.bm_sp }}</span>
                {% endif %}

                <input type="hidden" name="view" value="{{ v }}">

                <label class="toggle">
                  <span>Show tickers</span>
                  <span class="switch">
                    <input type="checkbox" name="show_tickers" value="1"
                           onchange="this.form.submit()"
                           {% if request.form.get('show_tickers') %}checked{% endif %}>
                    <span class="slider"></span>
                  </span>
                </label>
              </div>
              <div class="plot-wrap"
                    data-xmin="{{ '%.8f' % results.x_min }}"
                    data-xmax="{{ '%.8f' % results.x_max }}"
                    data-ymin="{{ '%.8f' % results.y_min }}"
                    data-ymax="{{ '%.8f' % results.y_max }}"
                    data-left="{{ '%.6f' % results.ax_left }}"
                    data-bottom="{{ '%.6f' % results.ax_bottom }}"
                    data-width="{{ '%.6f' % results.ax_width }}"
                    data-height="{{ '%.6f' % results.ax_height }}">
                <div class="coord-badge" id="coords">σ: — · μ: —</div>
                <img alt="plot" class="plot" id="plotImg" src="data:image/png;base64,{{ results.plot_b64 }}">
            </div>
          </div>
        </div>
      {% endif %}
    </form>
  </div>
<script>
(function(){
  const wrap  = document.querySelector('.plot-wrap');
  const img   = document.getElementById('plotImg');
  const badge = document.getElementById('coords');
  if(!wrap || !img || !badge) return;

  const xMin = parseFloat(wrap.dataset.xmin),
        xMax = parseFloat(wrap.dataset.xmax),
        yMin = parseFloat(wrap.dataset.ymin),
        yMax = parseFloat(wrap.dataset.ymax);

  // axes box (fração da figura) vindo do servidor
  const axL = parseFloat(wrap.dataset.left),
        axB = parseFloat(wrap.dataset.bottom),
        axW = parseFloat(wrap.dataset.width),
        axH = parseFloat(wrap.dataset.height);

  function fmt(n){
    if(!isFinite(n)) return '—';
    const a = Math.abs(n);
    return (a >= 0.001 && a < 1000) ? n.toFixed(3) : n.toExponential(2);
  }

  function update(e){
    const r = img.getBoundingClientRect();

    // converter frações -> px dentro da imagem
    const leftPx = axL * r.width;
    const topPx  = (1 - (axB + axH)) * r.height; // topo da área do gráfico
    const plotW  = axW * r.width;
    const plotH  = axH * r.height;

    // posição do rato dentro da área útil (clamp)
    const px = Math.min(Math.max(e.clientX - r.left - leftPx, 0), plotW);
    const py = Math.min(Math.max(e.clientY - r.top  - topPx , 0), plotH);

    // px -> dados (y invertido)
    const x = xMin + (px / plotW) * (xMax - xMin);
    const y = yMax - (py / plotH) * (yMax - yMin);

    badge.textContent = `σ: ${fmt(x)} · μ: ${fmt(y)}`;
  }

  img.addEventListener('mousemove', update);
  img.addEventListener('mouseenter', e => { update(e); badge.style.opacity = 1; });
  img.addEventListener('mouseleave', () => { badge.style.opacity = 0; });
})();
</script>
</body>
</html>

"""

@app.route("/", methods = ["GET", "POST"])

def index():
    
    error = None
    results = None

    if request.method =="POST":
        
        tickers = [t.strip().upper() for t in request.form.get("tickers", "").split(",") if t.strip()]
        benchmark =request.form.get("benchmark", "").strip()

        wr_raw = request.form.get("wanted_return", "").strip()
        wv_raw = request.form.get("wanted_volatility", "").strip()
        rf_raw = request.form.get("rf", "").strip()

        wanted_return = (float(wr_raw)/100 if wr_raw else "blank")
        wanted_volatility = (float(wv_raw)/100 if wv_raw else None)
        rf = float(rf_raw)/100 if rf_raw else 0.0

        bm_point = (compute_benchmark(benchmark, rf)[0],compute_benchmark(benchmark, rf)[1]) if benchmark else None

        view = (request.form.get("view") or "mvp").strip().lower()
        show_tickers = bool(request.form.get("show_tickers"))

        try:
            
          all_results = portfolios(tickers, wanted_return, wanted_volatility, rf)

          fig = graph(tickers, wanted_return, wanted_volatility, rf, view, benchmark=bm_point, show_tickers=show_tickers, benchmark_label=benchmark)
          plt.figure(fig.number)
          x_min, x_max = plt.xlim()
          y_min, y_max = plt.ylim()
          left, bottom, width, height = plt.gca().get_position().bounds
          buf = io.BytesIO()
          fig.savefig(buf, format="png", dpi=140, transparent= True)
          plt.close(fig)
          img_b64 = base64.b64encode(buf.getvalue()).decode("ascii")

          def portfolio_table(tickers, portfolio_weigths):
              weights = np.asarray(portfolio_weigths).reshape(-1)
              return [(n, float(w)) for n, w in zip(tickers, weights)]

          results = {}

          results["tickers"] = tickers
          results["rf"] = rf
          results["target_mu"] = (float(wanted_return) if (isinstance(wanted_return, (int, float)) or (isinstance(wanted_return, str) and wanted_return.strip().lower() != "blank")) else float(all_results[14]))
          results["target_sigma"] = ("-" if (isinstance(wanted_return, str) and wanted_return.strip().lower() != "blank") else (f"{wanted_volatility:.4f}" if wanted_volatility is not None else "-"))
          results["view"] = view

          results["mvp_weights"] = portfolio_table(tickers, all_results[0])
          results["mvp_r"] = float(all_results[1])
          results["mvp_s"] = float(all_results[2])

          results["tangent_weights"] = portfolio_table(tickers, all_results[7])
          results["tangent_r"] = float(all_results[8])
          results["tangent_s"] = float(all_results[9])

          results["opt_norf_weights"] = portfolio_table(tickers, all_results[3])
          results["opt_norf_r"] = float(all_results[14])
          results["opt_norf_s"] = float(all_results[4])

          results["opt_rf_weights"] = portfolio_table(tickers, all_results[10]) + [("CASH", all_results[11])]
          results["opt_rf_r"] = float(all_results[15])
          results["opt_rf_s"] = float(all_results[12])

          results["plot_b64"] = img_b64

          results["mvp_sp"] = float(all_results[16])
          results["op_sp"] = float(all_results[17])
          results["tp_sp"] = float(all_results[18])

          results["bm_sp"] = float(compute_benchmark(benchmark, rf)[2]) if benchmark else None

          results["x_min"] = float(x_min); results["x_max"] = float(x_max)
          results["y_min"] = float(y_min); results["y_max"] = float(y_max)
          results["ax_left"] = float(left); results["ax_bottom"] = float(bottom)
          results["ax_width"] = float(width); results["ax_height"] = float(height)

        
        except Exception as e:
            error = str(e)
  
    return render_template_string(PAGE, results=results, error=error)

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000, debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
